In [26]:
import pandas as pd
import pickle
import nltk
from nltk.corpus import stopwords
import re
from  nltk.stem.porter import PorterStemmer
from tqdm import tqdm
from sklearn.metrics import classification_report

In [27]:
df = pd.read_csv("Datasets/test_data.csv")

In [28]:
df.shape

(3576, 17)

In [29]:
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function
0,16996,EXCELLENT ER RN Opportunity Available Now,"US, IL, Urbana",NaN,NaN,NaN,"Our client, located in Urban, IL, is looking f...",NaN,NaN,0,1,0,Full-time,NaN,NaN,Hospital & Health Care,Health Care Provider
1,9358,Scrum Master / Website Development Project Man...,"US, FL, Tampa",NaN,NaN,352 Inc. is a full-service digital agency crea...,Other agencies may call this job “Project Mana...,Qualifications2-10 years of experience in webs...,What You’ll GetFreedom: We trust you to do you...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Project Management
2,11562,HR Assistant - Contract,"AU, NSW, Sydney",People & Culture,NaN,Squiz is one of the world's leading web soluti...,Squiz is an Australian owned and now multinati...,You could be a graduate or have many years of ...,NaN,0,1,0,Contract,Not Applicable,Unspecified,Information Technology and Services,Human Resources
3,1106,Regional Sales Director South Africa,"ZA, GT, Johannesburg",Sales,NaN,Upstream’s mission is to revolutionise the way...,The Regional Sales Director SA will help deriv...,Knowledge/Skills/ExperienceProven sales and ac...,"Includes attractive competitive base salary, c...",0,1,1,Full-time,Director,NaN,Telecommunications,Sales
4,1981,Petrophysicist,"US, OK, Oklahoma City",NaN,NaN,Valor Services provides Workforce Solutions th...,About the CompanyThis is an amazing job opport...,"Education: Bachelor’s degree in Geology, Geoph...","Industry, Location and CompensationIndustry: E...",0,1,0,Full-time,Associate,Bachelor's Degree,Oil & Energy,NaN


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3576 entries, 0 to 3575
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               3576 non-null   int64 
 1   title                3576 non-null   object
 2   location             3510 non-null   object
 3   department           1304 non-null   object
 4   salary_range         585 non-null    object
 5   company_profile      2940 non-null   object
 6   description          3576 non-null   object
 7   requirements         3012 non-null   object
 8   benefits             2167 non-null   object
 9   telecommuting        3576 non-null   int64 
 10  has_company_logo     3576 non-null   int64 
 11  has_questions        3576 non-null   int64 
 12  employment_type      2862 non-null   object
 13  required_experience  2201 non-null   object
 14  required_education   1970 non-null   object
 15  industry             2599 non-null   object
 16  functi

In [31]:
new_df=df.drop(columns=['job_id', 'location','department','salary_range', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience','required_education', 'industry', 'function'], axis=1)

In [32]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3576 entries, 0 to 3575
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            3576 non-null   object
 1   company_profile  2940 non-null   object
 2   description      3576 non-null   object
 3   requirements     3012 non-null   object
 4   benefits         2167 non-null   object
dtypes: object(5)
memory usage: 139.8+ KB


In [33]:
new_df.isnull().sum()

title                 0
company_profile     636
description           0
requirements        564
benefits           1409
dtype: int64

In [34]:
new_df["company_profile"].fillna(new_df["company_profile"].mode()[0], inplace=True)
new_df["requirements"].fillna(new_df["requirements"].mode()[0], inplace=True)
new_df["benefits"].fillna(new_df["benefits"].mode()[0], inplace=True)

C:\Users\perug\AppData\Local\Temp\ipykernel_26836\1094426381.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  new_df["company_profile"].fillna(new_df["company_profile"].mode()[0], inplace=True)
C:\Users\perug\AppData\Local\Temp\ipykernel_26836\1094426381.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values

In [35]:
new_df.isnull().sum()

title              0
company_profile    0
description        0
requirements       0
benefits           0
dtype: int64

In [36]:
new_df['merged_data'] = new_df["title"]+" "+new_df["company_profile"]+" "+new_df['description']+" "+new_df["requirements"]+new_df["benefits"]

In [37]:
print(new_df['merged_data'])

0       EXCELLENT ER RN Opportunity Available Now We P...
1       Scrum Master / Website Development Project Man...
2       HR Assistant - Contract Squiz is one of the wo...
3       Regional Sales Director South Africa Upstream’...
4       Petrophysicist  Valor Services provides Workfo...
                              ...                        
3571    Walkin for Sales / Marketing Specialist  We Pr...
3572    Home Health CNA--Arco, ID Critical Nurse Staff...
3573    Software Engineer (Senior and Junior) We belie...
3574    Work with us BillGuard is a personal finance s...
3575    OF | UX DESIGNER, MOBILE APP @OP !!! Kettle is...
Name: merged_data, Length: 3576, dtype: object


In [38]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stop_words)

{'mustn', 'as', 'until', 'an', "hadn't", 'where', 'you', 'each', 'between', "they've", "aren't", 'hadn', "it'd", "i'll", 'once', 'i', 'ourselves', 'own', 'their', 'both', "we'll", 'do', "couldn't", 'of', 'won', 'them', 'are', 'why', "that'll", 'yours', 'off', 'hasn', 'our', 'by', 'any', "it'll", 'herself', 'before', "i'm", 've', 'because', 's', "shan't", 'there', 'into', 'not', 'does', 'out', 'above', 'shan', 'very', 'didn', 'under', 'ain', 'than', 'through', 'shouldn', 'just', 'm', "wasn't", "hasn't", 'yourselves', 'most', "they'll", "they'd", 'nor', 'your', 'such', 'other', "you've", 'all', 'but', 'did', 'about', "he's", 'himself', 'few', "weren't", 'y', 'can', "we've", "isn't", "needn't", 'aren', 'so', 'when', "she's", "mightn't", 'has', 'my', "should've", 'doing', 'down', 'myself', 'again', 'over', 'themselves', 'ours', 'needn', 'were', 'only', 'then', 'too', 'after', "you're", 'during', 'whom', 'how', 'had', 'up', 'itself', 'couldn', 'and', 'isn', 'is', 'hers', 'a', 'from', "i'd",

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\perug\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
port_stem = PorterStemmer()

In [40]:
def preprocess_text(text):
    return re.sub(r'[^a-zA-Z]', " ", text).lower()

In [41]:
def stemming(content):
        words = content.split()  # Tokenize words
        return " ".join([port_stem.stem(word) for word in words if word not in stop_words])

In [42]:
tqdm.pandas()

In [43]:
new_df['merged_data'] = new_df['merged_data'].progress_apply(lambda x:stemming(preprocess_text(x)))

100%|██████████| 3576/3576 [00:14<00:00, 250.76it/s]


In [44]:
X = new_df["merged_data"].values

In [51]:
print(X)

['excel er rn opportun avail provid full time perman posit mani medium larg us compani interest find recruit high qualiti candid engin manufactur highli technic non technic job client locat urban il look er rn becom member team asap client institut high qualiti care medic research educ serv peopl client healthcar leader central illinoi level trauma center bed magnet hospit client employe reflect organ valu integr collabor account respect excel effort improv health peopl serv provid world class healthcar work environ fulli integr epic electron medic recordsst art facilitiestuit assist continu medic educ opportun full part time employeesform mentor program avail eas employe transitionsmagnet design sinc incent join team sign bonu limit time offer reloc mile reloc mile enhanc rn compens plan base year experi enhanc paid time hour base cumul year rn experi profession nurs advanc program design recogn reward regist nurs profession contribut work perform need must year ed experienceexperi ed

In [52]:
vectorizer = pickle.load(open("models/xgboost_vectorizer.pkl", "rb"))
model = pickle.load(open("models/xg_boostmodel.pkl", "rb"))

In [53]:
X_transformed = vectorizer.transform(X)

In [54]:
y_pred = model.predict(X_transformed)

In [57]:
print(y_pred)

[0 0 0 ... 0 0 0]
